In [2]:
!pip install gradio

  Using cached aiofiles-24.1.0-py3-none-any.whl.metadata (10 kB)
  Using cached audioop_lts-0.2.2-cp313-abi3-macosx_11_0_arm64.whl.metadata (2.0 kB)
  Using cached groovy-0.1.2-py3-none-any.whl.metadata (6.1 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached python_multipart-0.0.20-py3-none-any.whl.metadata (1.8 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl.metadata (9.7 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 2.2 MB/s eta 0:00:0000:0100:01m
Using cached aiofiles-24.1.0-py3-none-any.whl (15 kB)
Using cached audioop_lts-0.2.2-cp313-abi3-macosx_11_0_arm64.whl (26 kB)
Using cached groovy-0.1.2-py3-none-any.whl (14 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 2.8 MB/s eta 0:00:00a 0:00:01
Using cached semantic_version-2.10.0-py2.py3-none-any.whl (15 kB)
Using cached typing_extensions-4.15.0-py3-none-any.whl (44 kB)
   ━━━━━━━━━━━

In [4]:
!pip install pycountry

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 3.5 MB/s eta 0:00:00a 0:00:01


In [6]:
"""
---
title: World Bank Data Explorer Dashboard
emoji: 🌍
colorFrom: green
colorTo: purple
sdk: gradio
license: apache-2.0
---
"""
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gradio as gr

# Load data
DATA_PATH = "world_data.csv"
df = pd.read_csv(DATA_PATH)

# Map ISO Alpha-2 to Alpha-3 codes
continent_to_alpha2 = {
    "Africa": ["DZ","AO","BJ","BW","BF","BI","CM","CV","CF","TD","KM","CG","CD",
               "CI","DJ","EG","GQ","ER","SZ","ET","GA","GM","GH","GN","GW","KE","LS","LR","LY",
               "MG","MW","ML","MR","MU","MA","MZ","NA","NE","NG","RW","ST","SN","SC","SL","SO",
               "ZA","SS","SD","TZ","TG","TN","UG","ZM","ZW","GH","SS"],
    "Asia": ["AF","AM","AZ","BH","BD","BT","BN","KH","CN","CY","GE","IN","ID","IR","IQ","IL","JP",
             "JO","KZ","KW","KG","LA","LB","MY","MV","MN","MM","NP","KP","OM","PK","PS","PH",
             "QA","SA","SG","KR","LK","SY","TW","TJ","TH","TL","TR","TM","AE","UZ","VN","YE"],
    "Europe": ["AL","AD","AT","BY","BE","BA","BG","HR","CZ","DK","EE","FO","FI","FR","DE","GI",
               "GR","HU","IS","IE","IM","IT","XK","LV","LI","LT","LU","MT","MD","MC","ME","NL",
               "MK","NO","PL","PT","RO","RU","SM","RS","SK","SI","ES","SE","CH","UA","GB","VA"],
    "North America": ["AG","BS","BB","BZ","CA","CR","CU","DM","DO","SV","GD","GT","HT","HN","JM",
                      "MX","NI","PA","KN","LC","VC","TT","US"],
    "South America": ["AR","BO","BR","CL","CO","EC","FK","GF","GY","PY","PE","SR","UY","VE"],
    "Oceania": ["AS","AU","CK","FJ","PF","GU","KI","MH","FM","NR","NC","NZ","NU","NF","MP","PW",
                 "PG","WS","SB","TK","TO","TV","VU","WF"]
}

# Convert Alpha-2 to Alpha-3 using pycountry
import pycountry

def alpha2_to_alpha3(code):
    try:
        return pycountry.countries.get(alpha_2=code).alpha_3
    except:
        return None

continent_to_alpha3 = {
    cont: list(filter(None, [alpha2_to_alpha3(c) for c in codes]))
    for cont, codes in continent_to_alpha2.items()
}

# Create mapping of Country Code to Country Name
code_to_country = df[['Country Code', 'Country Name']].drop_duplicates().set_index('Country Code')['Country Name'].to_dict()

# Get country list for continent
def get_countries_by_continent(continent):
    codes = continent_to_alpha3.get(continent, [])
    countries = [code_to_country[c] for c in codes if c in code_to_country]
    return gr.update(choices=sorted(countries), value=None)

# Plotting function
def plot_indicator(continent, country, indicator, start_year, end_year):
    valid_codes = continent_to_alpha3.get(continent, [])
    country_code = next((k for k, v in code_to_country.items() if v == country), None)

    if not country_code or country_code not in valid_codes:
        fig, ax = plt.subplots()
        ax.text(0.5, 0.5, "Invalid country for selected continent.", ha='center', va='center')
        ax.axis("off")
        return fig

    try:
        country_df = df[(df["Country Code"] == country_code) & (df["Indicator Name"] == indicator)]
        series = country_df.loc[:, "1960":"2023"].T
        series.columns = [indicator]
        series.index = series.index.astype(int)
        series = series.loc[start_year:end_year]

        fig = series.plot(figsize=(8, 4), legend=False, title=f"{indicator} for {country}").get_figure()
        return fig
    except Exception:
        fig, ax = plt.subplots()
        ax.text(0.5, 0.5, "No data available for this selection.", ha='center', va='center')
        ax.axis("off")
        return fig

# Build Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("# 🌍 Global Indicators Dashboard")

    with gr.Row():
        continent_dropdown = gr.Dropdown(label="Select Continent", choices=list(continent_to_alpha3.keys()))
        country_dropdown = gr.Dropdown(label="Select Country")

    with gr.Row():
        indicator_dropdown = gr.Dropdown(label="Select Indicator", choices=df["Indicator Name"].dropna().unique().tolist())
        start_slider = gr.Slider(1960, 2023, value=1960, step=1, label="Start Year")
        end_slider = gr.Slider(1960, 2023, value=2023, step=1, label="End Year")

    plot_output = gr.Plot(label="Plot")

    continent_dropdown.change(fn=get_countries_by_continent, inputs=continent_dropdown, outputs=country_dropdown)
    plot_button = gr.Button("Show Plot")
    plot_button.click(fn=plot_indicator,
                      inputs=[continent_dropdown, country_dropdown, indicator_dropdown, start_slider, end_slider],
                      outputs=plot_output)

if __name__ == "__main__":
    demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
